测试版本，思路就是利用 DFS。

In [72]:
equations = [["a","b"],["c","d"]]
values = [1.0,1.0]
queries = [["a","c"],["b","d"],["b","a"],["d","c"]]
eq_value = [equations[i] + [values[i]] for i in range(len(equations))]

# 通过 equations 构造图
graph = {}
for x,y,z in eq_value:
    graph.setdefault(x, {x: 1})
    graph.setdefault(y, {y: 1})
    graph[x][y] = z
    graph[y][x] = 1/z


def DFS(graph, s, t):
    stack = []
    stack.append(s)
    vertex = s
    seen = set()
    seen.add(s)
    parent = {s:s}
    
    while vertex != t and len(stack)> 0:
        vertex = stack.pop()
        nodes = graph[vertex].keys()
        for v in nodes:
            if v not in seen:
                stack.append(v)
                seen.add(v)
                parent[v] = vertex
    return(parent)


final = []
for x, y in queries:
    if x not in graph.keys() or y not in graph.keys():
        final.append(-1)
    elif x == y:
        final.append(1)
    else:
        parent = DFS(graph, x, y)
        num = 1
        
        
        if y in parent.keys():
            while parent[y] != y:
                num = num * graph[parent[y]][y]
                y = parent[y]
            final.append(num)
        else:
            final.append(-1)
        
print(final)

[-1, -1, 1.0, 1.0]


执行用时 : 56 ms, 在Evaluate Division的Python3提交中击败了3.33% 的用户
内存消耗 : 13.2 MB, 在Evaluate Division的Python3提交中击败了0.00% 的用户

In [60]:
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        # 构造一个新的字典，里面是三元组，（除数，被除数，值）
        eq_value = [equations[i] + [values[i]] for i in range(len(equations))]
        # 构造一个图，键是给定的参数，值是与它有关系的参数及值
        graph = {}
        for x,y,z in eq_value:
            graph.setdefault(x, {x: 1})
            graph.setdefault(y, {y: 1})
            graph[x][y] = z
            graph[y][x] = 1/z
        
        # 利用深度优先算法看两个参数之间是否有一条路，如果有路则终止并返回这条路径，否则返回这个连通分支的所有搜索结果
        def DFS(graph, s, t):
            stack = []
            stack.append(s)
            vertex = s
            seen = set()
            seen.add(s)
            parent = {s:s}

            while vertex != t and len(stack)> 0:
                vertex = stack.pop()
                nodes = graph[vertex].keys()
                for v in nodes:
                    if v not in seen:
                        stack.append(v)
                        seen.add(v)
                        parent[v] = vertex             
            return(parent)

        # 最终结果放在一个数组中
        final = []
        for x, y in queries:
            # 如果有 1 个参数不在我们构造的图的键值中，则无法求值，加入 -1.0
            if x not in graph.keys() or y not in graph.keys():
                final.append(-1.0)
            # 相同的参数做除法，得到的值是 1.0
            elif x == y:
                final.append(1.0)
            else:
                # 除上述两种情况外，利用 DFS 算法 
                parent = DFS(graph, x, y)
                
                # 如果第二个参数在键值中，则每找一步就做一次乘法
                num = 1
                if y in parent.keys():
                    while parent[y] != y:
                        num = num * graph[parent[y]][y]
                        y = parent[y]
                    final.append(num)                
                # 不在键值中值是 -1.0
                else:
                    final.append(-1.0)

        return(final)

6.0


https://leetcode.com/problems/evaluate-division/discuss/180722/100-Clean-Python

In [ ]:
from collections import defaultdict
import collections
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        # 1, Graph Construction, N = (next, ratio)
        N = defaultdict(list)
        for (i, j), v in zip(equations, values):
            N[i] += (j, 1 / v),; N[j] += (i, v),

        # 2, Pick a root for each component, set root value to 1
        #    and compute the rest values
        P, V = {}, {}
        remain = set(N.keys())
        while remain:
            root = remain.pop()
            check = collections.deque([(root, 1)])
            while check:
                node, val = check.popleft()
                P[node], V[node] = root, val
                check += [(n, val * r) for n, r in N[node] if n in remain]
                remain -= {n for n, _ in N[node]}

        # 3, Query
        res = []
        for q1, q2 in queries:
            try:    res += [[-1, V[q1] / V[q2]][P[q1] == P[q2]]]
            except: res += [-1]
        return res

https://leetcode.com/problems/evaluate-division/discuss/88196/Simple'n'Clean-DFS-solution-in-Python

In [ ]:
class Solution:
    def calcEquation(self, equations: List[List[str]], values: List[float], queries: List[List[str]]) -> List[float]:
        def dfs(start, end, path, paths):
            if start == end and start in G:
                paths[0] = path
                return
            if start in vis: 
                return
            vis.add(start)
            for node in G[start]:
                dfs(node, end, path * W[start, node], paths)
        
        
        G, W = collections.defaultdict(set), collections.defaultdict(float)
        for (A, B), V in zip(equations, values):
            G[A], G[B] = G[A] | {B}, G[B] | {A}
            W[A, B], W[B, A] = V, 1.0 / V
            
        res = []
        for X, Y in queries:
            paths, vis = [-1.0], set()
            dfs(X, Y, 1.0, paths)
            res += paths[0],
        return res